**INSTALLATION**

In [ ]:
!git clone --recurse-submodules https://github.com/safednn-nasa/ProphecyPlus

Cloning into 'ProphecyPlus'...
remote: Enumerating objects: 2791, done.
remote: Counting objects: 100% (200/200), done.
remote: Compressing objects: 100% (172/172), done.
remote: Total 2791 (delta 135), reused 22 (delta 22), pack-reused 2591 (from 3)
Receiving objects: 100% (2791/2791), 70.22 MiB | 40.51 MiB/s, done.
Resolving deltas: 100% (1378/1378), done.


In [ ]:
%cd ./ProphecyPlus
!pwd
!ls -lt /content/ProphecyPlus/dataset_models/acasxu/

/content/ProphecyPlus
/content/ProphecyPlus
total 348
-rw-r--r-- 1 root root 123872 Dec 28 03:39 acasxu_new.h5
-rw-r--r-- 1 root root  67774 Dec 28 03:39 cnn_max_mninst2.onnx
-rw-r--r-- 1 root root     13 Dec 28 03:39 readme.txt
-rw-r--r-- 1 root root  55889 Dec 28 03:39 ACASXU_experimental_v2a_1_1.onnx
-rw-r--r-- 1 root root  96680 Dec 28 03:39 acasxu.h5


In [ ]:
import math
import io
import os
from collections import namedtuple
import sys
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn import tree
from tqdm import tqdm
import operator
import pandas as pd
#tf.compat.v1.disable_eager_execution()
#tf.compat.v1.enable_eager_execution()
print(tf.__version__)

2.15.0


In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 105.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 107.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.7/33.7 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.9/489.9 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 25.4 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing 

In [ ]:
! pip install onnx
! pip install onnx-tf

! pip install pybind11
! pip install cmake

import torch


import onnx
!pip install onnxruntime
import onnxruntime

#import keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 88.5 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/lib/python3.10/encodings/cp437.py", line 14, in decode
    def decode(self,input,errors='strict'):
KeyboardInterrupt

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packa

**GET MODEL AND DATA**

In [ ]:
acas_train = np.empty([384221,5],dtype=float)
acas_train_labels = np.zeros(384221,dtype=int)

def read_inputs_from_file(inputFile):
    global acas_train, acas_train_labels, num
    with open(inputFile) as f:
        lines = f.readlines()
        print(len(lines), "examples")
        acas_train = np.empty([len(lines),5],dtype=float)
        acas_train_labels = np.zeros(len(lines),dtype=int)

        for l in range(len(lines)):
            k = [float(stringIn) for stringIn in lines[l].split(',')]
            if len(k) > 5:
              lab = int(k[5])
              acas_train_labels[l+num] = lab

            count = 0
            for i in range(0,5):
                acas_train[l+num][i] = k[i]

!wget https://raw.githubusercontent.com/safednn-nasa/prophecy_DNN/master/clusterinACAS_0_short.csv -O ./clusterinACAS_0_shrt.csv

num = 0


read_inputs_from_file('./clusterinACAS_0_shrt.csv')



print((acas_train).shape)


train_len = (len(acas_train) * 0.75)
train_data = acas_train[:int(train_len)]
train_labels = acas_train_labels[:int(train_len)]

test_data = acas_train[int(train_len):]
test_labels = acas_train_labels[int(train_len):]

print((train_data).shape)
print((train_labels).shape)
print((test_data).shape)
print((test_labels).shape)




In [ ]:
from tensorflow import keras

x_train = np.array(train_data)
y_train = np.eye(5)[train_labels]
y_train1 = tf.argmax(y_train, 1)

x_test = np.array(test_data)
y_test = np.eye(5)[test_labels]
y_test1 = tf.argmax(y_test, 1)

print(np.shape(x_train))
print(np.shape(x_test))
print(np.shape(y_train))
print(np.shape(y_test))


np.save('./x_train_npy.npy', x_train)
np.save('./x_test_npy.npy', x_test)
np.save('./y_train_npy.npy', y_train1)
np.save('./y_test_npy.npy', y_test1)


model=keras.models.load_model('/content/ProphecyPlus/dataset_models/acasxu/acasxu_new.h5')
x_train_rshape = x_train.reshape(-1, 1, 1, 5)
x_test_rshape = x_test.reshape(-1, 1, 1, 5)
print(np.shape(x_train_rshape))
print(np.shape(x_test_rshape))
np.save('./x_train_rshape.npy', x_train_rshape)
np.save('./x_test_rshape.npy', x_test_rshape)

y_predict = model.predict(x_train_rshape)
y_predict1 = tf.argmin(y_predict, 1)
np.save('./y_predict_npy.npy', y_predict1)

correct_prediction1 = tf.equal(tf.argmin(y_predict, 1), tf.argmax(y_train, 1))
accuracy1 = tf.reduce_mean(tf.cast(correct_prediction1, tf.float32))
print(accuracy1.numpy())

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/lambda_layer.py:327: UserWarning: onnx2keras.utils is not loaded, but a Lambda layer uses it. It may cause errors.
  function = cls._parse_function_from_config(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/lambda_layer.py:327: UserWarning: onnx2keras.elementwise_layers is not loaded, but a Lambda layer uses it. It may cause errors.
  function = cls._parse_function_from_config(


(288165, 5)
(96056, 5)
(288165, 5)
(96056, 5)
Tensor("Placeholder:0", shape=(None, 50), dtype=float32) Tensor("Placeholder_1:0", shape=(50,), dtype=float32)
Tensor("Placeholder:0", shape=(None, 50), dtype=float32) Tensor("Placeholder_1:0", shape=(50,), dtype=float32)
Tensor("Placeholder:0", shape=(None, 50), dtype=float32) Tensor("Placeholder_1:0", shape=(50,), dtype=float32)
Tensor("Placeholder:0", shape=(None, 50), dtype=float32) Tensor("Placeholder_1:0", shape=(50,), dtype=float32)
Tensor("Placeholder:0", shape=(None, 50), dtype=float32) Tensor("Placeholder_1:0", shape=(50,), dtype=float32)
Tensor("Placeholder:0", shape=(None, 50), dtype=float32) Tensor("Placeholder_1:0", shape=(50,), dtype=float32)
Tensor("Placeholder:0", shape=(None, 5), dtype=float32) Tensor("Placeholder_1:0", shape=(5,), dtype=float32)
(288165, 1, 1, 5)
(96056, 1, 1, 5)
Tensor("model_6/Operation_1_MatMul/MatMul:0", shape=(None, 50), dtype=float32) Tensor("model_6/Operation_1_Add_const2/Const:0", shape=(50,), dty

**INVOKE PROPHECY TO EXTRACT RULES**

In [ ]:
!python -m prophecy.main -m '/content/drive/MyDrive/prophecy/acasxu_new.h5' -wd '/content/ProphecyPlus/results/mnist/rules_all/' analyze -h

usage: main.py analyze [-h] -tx TRAIN_FEATURES -ty TRAIN_LABELS -vx VAL_FEATURES -vy VAL_LABELS
                       [-odl] [-oal] [-sr] [-b] [-c] [-rs RANDOM_STATE] [-type TYPE]
                       [-inptype INPTYPE] [-acts ACTS] [-layer_name LAYER_NAME] [-top TOP]

options:
  -h, --help            show this help message and exit
  -tx TRAIN_FEATURES, --train_features TRAIN_FEATURES
                        Train features
  -ty TRAIN_LABELS, --train_labels TRAIN_LABELS
                        Train labels
  -vx VAL_FEATURES, --val_features VAL_FEATURES
                        Validation features
  -vy VAL_LABELS, --val_labels VAL_LABELS
                        Validation labels
  -odl, --only-dense-layers
                        Consider only dense layers
  -oal, --only-activation-layers
                        Include the activation layers associated to the dense layers
  -sr, --skip-rules     Skip rules extraction
  -b, --balance         Balance classes in the dataset for traini

**PARAMETERS: rules per label (based on given labels array), rules in terms of ON/OFF act values, TOP rules (highest train recall), just from layer "rule_3".**

In [ ]:
import os
import shutil

if os.path.exists('/results/acasxu/rules/'):
  shutil.rmtree('/results/acasxu/rules/')

os.makedirs('/results/acasxu/rules/')

!pwd
!python -m prophecy.main -m '/content/ProphecyPlus/dataset_models/acasxu/acasxu_new.h5' -wd '/content/ProphecyPlus/results/acasxu/rules/' analyze -tx ./x_train_rshape.npy -ty ./y_predict_npy.npy -vx ./x_test_rshape.npy -vy ./y_predict_npy.npy -layer 'relu_3' -type 3 -top True -acts True

In [ ]:
import pandas as pd
_output_path = "/content/ProphecyPlus/results/acasxu/rules/ruleset.csv"


print("****** RULES ********")
df_op = pd.read_csv(_output_path)
df_op


****** RULES ********


,layer,layer_count,neurons,signature,support,label,train_coverage,train_precision,train_recall,train_f1,test_coverage,test_precision,test_recall,test_f1,f1,kind
0,relu_3,1,"[45, 43, 25, 32, 21, 30, 16, 24, 35, 11, 10, 5]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",25989,0,9.018791,100,12.302718,21.909921,8.322229,71.778834,2.716264,5.234446,13.572183,correct
1,relu_3,1,"[45, 3, 11, 31, 15, 25, 30, 32, 28, 29, 18, 16...","[1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, ...",204,1,0.070793,100,1.698726,3.340703,0.005205,0.000000,0.000000,0.000000,1.670351,correct
2,relu_3,1,"[45, 3, 11, 31, 15, 23, 36, 0, 14, 43, 10, 30,...","[1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, ...",599,3,0.207867,100,2.317035,4.529129,0.260265,9.600000,0.092836,0.183894,2.356512,correct
3,relu_3,1,"[45, 43, 11, 25, 46, 24, 3, 8, 41, 35]","[0, 1, 1, 1, 1, 0, 1, 1, 0, 1]",489,4,0.169694,100,1.955140,3.835294,0.064546,14.516129,0.035984,0.071790,1.953542,correct
4,relu_3,1,"[45, 3, 11, 31, 36, 30, 20, 6, 32, 25, 4, 44, ...","[1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0]",92,2,0.031926,100,0.654944,1.301365,0.000000,0.000000,0.000000,0.000000,0.650683,correct


**Processing rules to obtain neuron patterns (neurons and their respective signatures)**

In [ ]:
print("****** RULES ********")
df_op = pd.read_csv(_output_path)

print("****** Label 0 RULES ********")
relu_3_rule_neurons = df_op[df_op['label'] == 0][df_op['layer'] == 'relu_3']['neurons']
relu_3_rule_signature = df_op[df_op['label'] == 0][df_op['layer'] == 'relu_3']['signature']


****** RULES ********
****** Label 0 RULES ********


<ipython-input-11-7246f7e71a25>:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  relu_3_rule_neurons = df_op[df_op['label'] == 0][df_op['layer'] == 'relu_3']['neurons']
<ipython-input-11-7246f7e71a25>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  relu_3_rule_signature = df_op[df_op['label'] == 0][df_op['layer'] == 'relu_3']['signature']


**PROOFS WITH MARABOU**

**MARABOU SETUP:**

IMPORTANT: INSTALL AND BUILD MARABOU from https://github.com/NeuralNetworkVerification/Marabou.

The following code assumes /content/drive/MyDrive/Marabou_bld to be the Marabou build directory

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os
os.chdir('/content/drive/MyDrive/Marabou_bld')
!pwd
!ls -lt

path = os.environ['PATH']
print(path)
#os.environ['PATH'] = '/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/content/Marabou/build/Marabou:/content/Marabou/build/Marabou/build:/content/Marabou/build/Marabou/build/bin:/content/Marabou/build/Marabou:/content/Marabou/build/Marabou/build:/content/Marabou/build/Marabou/build/bin'
print(os.environ['PATH'])
os.environ['PATH'] = path + ':/content/drive/MyDrive/Marabou_bld:/content/drive/MyDrive/Marabou_bld/build:/content/drive/MyDrive/Marabou_bld/build/bin'
print(os.environ['PATH'])

**prophecy helper methods**.

In [ ]:
def check_pattern(layer_vals: list, neuron_ids: list, neuron_sig: list) -> bool:
    """
        Check if the provided layer values satisfy the provided neuron signature.
    :param layer_vals:
    :param neuron_ids:
    :param neuron_sig:
    :return:
    """
    found = True
    oper = -1
    # layer_vals = (layer_vals).flatten()

    for ind in range(0, len(neuron_sig)):
        if ind % 2 == 0:
            op = neuron_sig[ind]
            if op == '<=':
                oper = 0
            else:
                oper = 1
        else:
            v = int(neuron_ids[(int)(ind / 2)])
            vsig = float(neuron_sig[ind])
            val = float(layer_vals[v])
            # print(v,vsig,val,oper)
            if oper == 0:
                if val > vsig:
                    # print(v,val,vsig,oper)
                    found = False
                    break
            else:
                if val <= vsig:
                    # print(v,val,vsig,oper)
                    found = False
                    break
            oper = -1

    return found

def get_suffix_cluster(neuron_ids, neuron_sig, suffixes, VAL=False):
    # Get the cluster of inputs that such that all inputs in the cluster
    # have provided on/off signature for the provided neurons.
    #
    # The returned cluster is an array of indices (into mnist.train.images).
    if (VAL == False):
        return np.where((suffixes[:, neuron_ids] == neuron_sig).all(axis=1))[0]

    matched_ids = []
    # print(len(suffixes))
    for indx in range(0, len(suffixes)):
        if (check_pattern(suffixes[indx], neuron_ids, neuron_sig) == True):
            matched_ids.append(indx)
    # print(matched_ids)
    return matched_ids



**Get fingerprints (neuron values) after relu_3 for train data.**

In [ ]:
from keras import backend

func3 = None


for layer in model.layers:
    print(layer.name)
    if (layer.name == 'relu_3'):
      func3 = backend.function(model.input, [layer.output])


fingerprint_3 = []

if (func3 != None):
  fingerprint_3 = func3(x_train_rshape)

**Get values for the lower and upper bounds for input variables and dense layer neuron outputs based on the rules.**

In [ ]:
print(len(x_train))
x_train_min = np.zeros(5)
x_train_max = np.zeros(5)

for indx in range(0,5):
  x_train_min[indx] = np.min(x_train[:,indx])
  x_train_max[indx] = np.max(x_train[:,indx])

print(x_train_min)
print(x_train_max)


print("RULE:")
###########
rule_neurons_list_3 = []
rule_neurons = (relu_3_rule_neurons.array[0]).split(",")
for indx in range(0, len(rule_neurons)):
    rule_neurons[indx] = (rule_neurons[indx]).strip()
    rule_neurons[indx] = (rule_neurons[indx]).replace("[", "")
    rule_neurons[indx] = (rule_neurons[indx]).replace("]","")
    rule_neurons_list_3.append(int(rule_neurons[indx]))

print(rule_neurons_list_3)

rule_sig_list_3 = []
rule_sig = (relu_3_rule_signature.array[0]).split(",")
for indx in range(0, len(rule_sig)):
    rule_sig[indx] = (rule_sig[indx]).strip()
    rule_sig[indx] = (rule_sig[indx]).replace("[", "")
    rule_sig[indx] = (rule_sig[indx]).replace("]","")
    rule_sig_list_3.append(int(rule_sig[indx]))

print(rule_sig_list_3)

fngprnt = (fingerprint_3[0] > 0.0).astype('int')
indices = get_suffix_cluster(rule_neurons_list_3, rule_sig_list_3, fngprnt)
print("indices:", len(indices))

x_train3 = []
fngprnt3 = []
for indx in range(0, len(indices)):
    if (indx == 0):
      print("INP:",x_train[indices[indx]])
      print("FINGER:",fingerprint_3[0][indices[indx]])
    x_train3.append(x_train[indices[indx]])
    fngprnt3.append(fingerprint_3[0][indices[indx]])

x_train3 = np.array(x_train3)
fngprnt3 = np.array(fngprnt3)

print(np.shape(x_train3))
x_train_min3 = np.zeros(5)
x_train_max3 = np.zeros(5)

for indx in range(0,5):
  x_train_min3[indx] = np.min(x_train3[:,indx])
  x_train_max3[indx] = np.max(x_train3[:,indx])

print(x_train_min3)
print(x_train_max3)

print(np.shape(fngprnt3))
fngprnt_min3 = np.zeros(len(fngprnt3[0]))
fngprnt_max3 = np.zeros(len(fngprnt3[0]))

for indx in range(0,len(fngprnt3[0])):
  fngprnt_min3[indx] = np.min(fngprnt3[:,indx])
  fngprnt_max3[indx] = np.max(fngprnt3[:,indx])

print(fngprnt_min3)
print(fngprnt_max3)


288165
[-0.320142 -0.5      -0.5      -0.5      -0.5     ]
[0.679858 0.5      0.5      0.5      0.25    ]
RULE:
[45, 43, 25, 32, 21, 30, 16, 24, 35, 11, 10, 5]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
indices: 25989
INP: [ 0.100096 -0.5      -0.5      -0.5      -0.5     ]
FINGER: [0.10179728 0.         1.3244011  0.04412816 0.57699263 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.39995432 0.         2.1758285
 0.         0.         0.         0.         3.7059731  1.4501623
 0.         0.         0.11942634 0.         0.         1.348339
 0.         0.16858709 0.         0.         0.         0.
 0.20186378 0.2259273  0.         0.         0.05198997 1.1146446
 0.         0.         0.         0.         0.         0.
 0.         0.25532976]
(25989, 5)
[-0.240207 -0.5      -0.5      -0.5      -0.5     ]
[0.679858 0.5      0.5      0.5      0.25    ]
(25989, 50)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0

In [ ]:
onnx_model = onnx.load("./resources/onnx/acasxu/ACASXU_experimental_v2a_1_1.onnx")
onnx.checker.check_model(onnx_model)

**Invoke Marabou to solve the following query**

**Prove rule_label0(x) /\ F(x) = l is UNSAT (for all l not equal to label0)**

In [ ]:
import os
import numpy as np
from maraboupy import Marabou
from maraboupy.MarabouCore import *
from maraboupy.MarabouPythonic import *

options = Marabou.createOptions(verbosity = 1, numWorkers=1, numBlasThreads=1,snc=True)

filename = "./resources/onnx/acasxu/ACASXU_experimental_v2a_1_1.onnx"
network_a = Marabou.read_onnx(filename)

#-0.295234 -0.5      -0.5      -0.5      -0.5
print("INPUT VARS")
invars = network_a.inputVars[0][0].flatten()
print(invars)
for i in invars:
    v = Var(i)
    if (i == 0):
      network_a.setLowerBound(i,x_train_min3[0])
      network_a.setUpperBound(i,x_train_max3[0])
      #network_a.setLowerBound(i,0.100096)
      #network_a.setUpperBound(i,0.100096)
    if (i == 1):
      network_a.setLowerBound(i,x_train_min3[1])
      network_a.setUpperBound(i,x_train_max3[1])
      #network_a.setLowerBound(i,-0.5)
      #network_a.setUpperBound(i,-0.5)
    if (i == 2):
      network_a.setLowerBound(i,x_train_min3[2])
      network_a.setUpperBound(i,x_train_max3[2])
      #network_a.setLowerBound(i,-0.5)
      #network_a.setUpperBound(i,-0.5)
    if (i == 3):
      network_a.setLowerBound(i,x_train_min3[3])
      network_a.setUpperBound(i,x_train_max3[3])
      #network_a.setLowerBound(i,-0.5)
      #network_a.setUpperBound(i,-0.5)
    if (i == 4):
      network_a.setLowerBound(i,x_train_min3[4])
      network_a.setUpperBound(i,x_train_max3[4])
      #network_a.setLowerBound(i,-0.5)
      #network_a.setUpperBound(i,-0.5)

print("LAYER VARS MAP")
print(network_a.layerNameToVariables)



dense_3_neurons = network_a.layerNameToVariables["relu_3"][0]
print(np.shape(dense_3_neurons))
print(len(dense_3_neurons))
print(dense_3_neurons[0])


inp = [0.10179728,0,1.3244011,0.04412816,0.57699263,0,0,0,0,0,0,0,0,0,0,0.39995432,0,2.1758285,0,0,0,0,3.7059731,1.4501623,0,0,0.11942634,0,0,1.348339,0,0.16858709,0,0,0,0,0.20186378,0.2259273,0,0,0.05198997,1.1146446,0,0,0,0,0,0,0,0.25532976]


for indx in range(0, len(dense_3_neurons)):
    neuron_indx = dense_3_neurons[indx] - dense_3_neurons[0]
   # print(neuron_indx)
    network_a.setLowerBound(dense_3_neurons[indx], fngprnt_min3[neuron_indx])
    network_a.setUpperBound(dense_3_neurons[indx], fngprnt_max3[neuron_indx])
    #network_a.setLowerBound(dense_3_neurons[indx], inp[neuron_indx] - 0.1)
    #network_a.setUpperBound(dense_3_neurons[indx], inp[neuron_indx] + 0.1)


print("OUTPUT VARS")
outvars = network_a.outputVars[0].flatten()
print(outvars)


rule_label = 0
prove = True
for label in range(0,  len(outvars)):
  if (label == rule_label):
    continue

  label_var = Var(outvars[label])
  for indx in range(0,  len(outvars)):
    v = Var(outvars[indx])
    if (indx == label):
      continue
    network_a.addConstraint(label_var >= v + 0.001)
    print(v, ":",indx)

  sat_unsat,vals, stats = network_a.solve(options = options)

  print("sat_unsat:", sat_unsat)

  if (sat_unsat == 'sat'):
    print("SAT for label:", label)
    print("vals:", vals)
    prove = False
    break
  else:
    print("UNSAT for label:", label)

if (prove == True):
   print("Rule Proved!!")


**COMPOSITIONAL PROOF DEMONSTRATION**

**NEED TO PROVE A => B on F(X)**

**A is a pre-condition on the input variables**

**B is condition on F(X), F(X) = COC (label 0)**

**Create F': small network from inner layer (relu_3) to output.**

**Create F'': small network from input to layer (relu_3).**

**(1) Prove rule_label0(X) => (F'(Z) = 0).**

**(2) Prove A => rule_label0 on F''(X).**

**Prove rule_label0(z) /\ F'(z) = l is UNSAT (for all l not equal to label0)**



In [ ]:
import os
import numpy as np
from maraboupy import Marabou
from maraboupy.MarabouCore import *
from maraboupy.MarabouPythonic import *



def set_solver_bounds(min_bnds,max_bnds,other_lbl):

  filename = "./resources/onnx/acasxu/ACASXU_experimental_v2a_1_1.onnx"
  network_dense3_op = Marabou.read_onnx(filename,inputNames=['relu_3'])
  #print("layer to vars:", network_dense3_op.layerNameToVariables)

  print("INPUT VARS")
  invars = network_dense3_op.inputVars[0][0].flatten()
  print(invars)
  for indx in range(0, len(invars)):
    neuron_indx = invars[indx]
    v = Var(invars[indx])
    network_dense3_op.setLowerBound(invars[indx],min_bnds[indx])
    network_dense3_op.setUpperBound(invars[indx],max_bnds[indx])


  print("OUTPUT VARS")
  outvars = network_dense3_op.outputVars[0].flatten()
  print(outvars)

  label_var = Var(outvars[other_lbl])
  label = other_lbl
  for indx in range(0, len(outvars)):
    v = Var(outvars[indx])
    if (indx == label):
      continue
    network_dense3_op.addConstraint(v >= label_var + 0.001)
    print(v, ":",indx)

  print(network_dense3_op.lowerBounds)
  print(network_dense3_op.upperBounds)
  return network_dense3_op

In [ ]:
import copy
options = Marabou.createOptions(verbosity = 1, numWorkers=1, numBlasThreads=1, snc = True)
min_bnds = copy.deepcopy(fngprnt_min3)
max_bnds = copy.deepcopy(fngprnt_max3)

other_lbl = 1


while (True):
  for indx1 in range(0, len(min_bnds)):
    print(indx1, min_bnds[indx1], max_bnds[indx1])
    assert(min_bnds[indx1] <= max_bnds[indx1])

  print("min_bnds:", min_bnds)
  print("max_bnds:", max_bnds)

  network = set_solver_bounds(min_bnds,max_bnds,other_lbl)
  sat_unsat,vals, stats = network.solve(options = options)
  print("sat_unsat:", sat_unsat)
  print("vals:", vals)
#print("stats:", stats)
  if (sat_unsat == "sat"):
    for indx in range(0, len(invars)):
      VAL = vals[invars[indx]]
      LB = network.lowerBounds[invars[indx]]
      UB = network.upperBounds[invars[indx]]
      if ( np.abs(VAL - LB) <= np.abs(VAL - UB)):
        new_val = LB + np.abs(VAL - LB) + 0.001
        if (new_val <= max_bnds[invars[indx]]):
          min_bnds[invars[indx]] = new_val
        else:
          min_bnds[invars[indx]] = max_bnds[invars[indx]]
      else:
        new_val = UB - np.abs(VAL - UB) - 0.001
        if (new_val >= min_bnds[invars[indx]]):
          max_bnds[invars[indx]] = new_val
        else:
          max_bnds[invars[indx]] = min_bnds[invars[indx]]
  else:
    break

print("LABEL PROVED:", other_lbl)





0 0.0 3.1667051315307617
1 0.0 3.630335807800293
2 0.0 1.8768987655639648
3 0.0 1.7701197862625122
4 0.0 4.023116111755371
5 0.0 0.0
6 0.0 0.8388819694519043
7 0.0 1.2758440971374512
8 0.0 0.7351728677749634
9 0.0 0.1352209448814392
10 0.0 0.0
11 0.0 0.0
12 0.0 2.656733274459839
13 0.0 0.7284724116325378
14 0.0 0.0881415605545044
15 0.0 0.7926082611083984
16 0.0 0.0
17 0.0 3.108473777770996
18 0.0 0.5346285700798035
19 0.0 0.0
20 0.0 1.5903397798538208
21 0.0 0.0
22 0.0 6.586481094360352
23 0.0 3.263119697570801
24 0.0 0.0
25 0.0 0.0
26 0.0 2.7367746829986572
27 0.0 1.2828859090805054
28 0.0 0.4888882637023926
29 0.0 2.741816520690918
30 0.0 0.0
31 0.0 0.5131329894065857
32 0.0 0.0
33 0.0 1.0103355646133423
34 0.0 1.1220282316207886
35 0.0 0.0
36 0.0 0.9471767544746399
37 0.0 0.5216124057769775
38 0.0 1.2853522300720215
39 0.0 0.07664245367050171
40 0.0 1.261654019355774
41 0.0 2.1199440956115723
42 0.0 3.5087649822235107
43 0.0 0.0
44 0.0 0.3370699882507324
45 0.0 0.0
46 0.0 3.3229994

**Prove A => rule_label0 on F''(X) (Work in progress)**

In [ ]:
import os
import numpy as np
from maraboupy import Marabou
from maraboupy.MarabouCore import *
from maraboupy.MarabouPythonic import *

options = Marabou.createOptions(verbosity = 1, numWorkers=1, numBlasThreads=1)

filename = "./resources/onnx/acasxu/ACASXU_experimental_v2a_1_1.onnx"
network_inp_dense3 = Marabou.read_onnx(filename,outputNames=['relu_3'])

print("INPUT VARS")
invars = network_inp_dense3.inputVars[0][0].flatten()
print(invars)
#### A (property 10) for COC
##[0.268978, 0.111408, -0.5, 0.227273, 0.0]
##[0.679858, 0.499999, -0.49841, 0.5, 0.5]
for i in invars:
    v = Var(i)
    if (i == 0):
      network_inp_dense3.setLowerBound(i,0.268978)
      network_inp_dense3.setUpperBound(i,0.679858)
    if (i == 1):
      network_inp_dense3.setLowerBound(i,0.111408)
      network_inp_dense3.setUpperBound(i,0.499999)
    if (i == 2):
      network_inp_dense3.setLowerBound(i,-0.5)
      network_inp_dense3.setUpperBound(i,-0.49841)
    if (i == 3):
      network_inp_dense3.setLowerBound(i,0.227273)
      network_inp_dense3.setUpperBound(i,0.5)
    if (i == 4):
      network_inp_dense3.setLowerBound(i,0.0)
      network_inp_dense3.setUpperBound(i,0.5)


print("OUTPUT VARS")
outvars = network_inp_dense3.outputVars[0].flatten()
print(outvars)

for v in outvars:
    neuron_indx = v
   # print(neuron_indx)
    if (neuron_indx in rule_neurons_list_3):
    #  v = Var(invars[indx])
      index = rule_neurons_list_3.index(neuron_indx)
      print("found:",index)
      print("Neuron:", neuron_indx, ",Act:", rule_sig_list_3[index])
      if (rule_sig_list_3[index] == 0):
        network_inp_dense3.setUpperBound(v,0)
        min_layer = np.min(fingerprint_3[0][:,neuron_indx])
        print(neuron_indx,"MIN:",min_layer)
        if (min_layer <= 0.0):
          network_inp_dense3.setLowerBound(v,min_layer)
      else:
        network_inp_dense3.setLowerBound(v,0)
        max_layer = np.max(fingerprint_3[0][:,neuron_indx])
        print(neuron_indx,"MAX:",max_layer)
        if (max_layer >= 0.0):
          network_inp_dense3.setUpperBound(v,max_layer)

      print(v, ":",v)


sat_unsat,vals, stats = network_inp_dense3.solve(options = options)

print("sat_unsat:", sat_unsat)
print("vals:", vals)
print("stats:", stats)

INPUT VARS
[0 1 2 3 4]
OUTPUT VARS
[260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277
 278 279 280 281 282 283 284 285 286 287 288 289 290 291 292 293 294 295
 296 297 298 299 300 301 302 303 304 305 306 307 308 309]
